<a href="https://www.kaggle.com/code/tmncollins/ml-journal-7-boiling-point-predictor?scriptVersionId=281498707" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
try:
    import rdkit
except:
    !pip install --quiet /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl

from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import rdkit

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

import networkx as nx
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdmolops
from rdkit import Chem
import pickle

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# IMPORT DATASET - FINGERPRINT
df_train = (pd.read_csv('/kaggle/input/smiles-boiling-point/solvents_bp.csv')).dropna()

df_save = df_train.dropna().drop_duplicates(subset=['smiles'])
df_save.to_csv('bp_data.csv')

In [ ]:
def canonical_smile(smile):
    molecule = Chem.MolFromSmiles(smile)
    canonical = Chem.MolToSmiles(molecule, canonical = True)
    return canonical

def compute_all_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    desc = []
    for d in Descriptors.descList:
        desc.append(d[1](mol))
    return desc

def compute_graph_features(smiles, graph_feats):
    mol = Chem.MolFromSmiles(smiles)
    adj = rdmolops.GetAdjacencyMatrix(mol)
    G = nx.from_numpy_array(adj)

    graph_feats['graph_diameter'].append(nx.diameter(G) if nx.is_connected(G) else 0)
    graph_feats['avg_shortest_path'].append(nx.average_shortest_path_length(G) if nx.is_connected(G) else 0)
    graph_feats['num_cycles'].append(len(list(nx.cycle_basis(G))))

def preprocessing(df):
    desc_names = [desc[0] for desc in Descriptors.descList]
    descriptors = [compute_all_descriptors(smile) for smile in df['smiles'].to_list()]

    graph_feats = {'graph_diameter': [], 'avg_shortest_path': [], 'num_cycles': []}
    for smile in df['smiles']:
        compute_graph_features(smile, graph_feats)

    result = pd.concat([
			pd.DataFrame(descriptors, columns = desc_names),
			pd.DataFrame(graph_feats)
		],
		axis = 1
	)
    result = result.replace([-np.inf, np.inf], np.nan)
    return result

# Visualising the Data

The boiling points (in K) and SMILES representations of 3790 diverse molecules were collected. The composition of the dataset is visualised below.

The distribution of boiling points is skewed with a tail towards higher boiling points.

In [ ]:
binwidth=10
data = df_train['bp_K']
plt.hist(data, bins=np.arange(min(data), max(data) + binwidth, binwidth), density=False)
plt.xlabel('Boiling Point / K')
plt.ylabel('Frequency')
plt.show() 

In [ ]:
alcohol = 0; ketone = 0; alkene = 0; halogen = 0; amine = 0; aromatic = 0; 
nitrile = 0; alkyne = 0; alkane = 0; sulfide = 0; 

for s in df_train['smiles']:
    if 'N' in s: amine += 1
    if 'O' in s: alcohol += 1
    if 'c' in s: aromatic += 1
    if 'C=C' in s: alkene += 1
    if '#N' in s or 'N#' in s: nitrile += 1
    elif '#' in s: alkyne += 1
    if 'Br' in s or 'F' in s or 'Cl' in s: halogen += 1
    if '=O' in s or 'O=' in s: ketone += 1
    if 'S' in s: sulfide += 1
    is_alkane = True
    for i in 'NOc=#FBlS':
        if i in s:
            is_alkane = False
            break
    if is_alkane: alkane += 1

labels = ['Alcohol', 'Amine', 'Alkane', 'Halide', 'Ketone', 'Alkene', 'Aromatic', 'Nitrile', 'Sulfide']
sizes = [alcohol, amine, alkane, halogen, ketone, alkene, aromatic, nitrile, sulfide]
fig, ax = plt.subplots()
ax.pie(sizes, labels=labels)
ax.plot()

# Morgan Fingerprints

Morgan extended connectivity fingerprints were constructed for each molecule using RDKit, using a radius of 3 and a length of 2048. Features that had a variance of less than 0.1% were dropped, resulting in a total of 1205 features for training. The correlation of these features with the boiling point is shown below.

Evidently, most features in the fingerprints are only weakly correlated with the boiling point.
The total data was split into a training and testing set in a 9:1 split. An XGBoost model was fitted to the training data. Its hyperparameters were tuned using a Randomised CV Search with 5-fold cross validation. 

The final model achieved a training score of 0.9470 and a testing score of 0.8117.
A Random Forest Regressor was also fitted to the training data using 50 estimators. This achieved higher training and testing scores of 0.9612 and 0.8325, respectively.

A plot of the prediction vs actual boiling points for the Random Forest model is given below. Lines have been drawn showing the +/- 10% confidence interval. 

In [ ]:
# IMPORT DATASET - FINGERPRINT
df_train = (pd.read_csv('/kaggle/input/smiles-boiling-point/bp_data.csv'))
print(len(df_train))

X = df_train.drop('bp_K', axis=1)
fingerprint_length = 2**11
fingerprint_cols = [[] for _ in range(fingerprint_length)]
for f in df_train['fingerprint']:
    f = str(f)
    zeros = fingerprint_length - len(f)
    f = ("0" * zeros) + f 
    for i in range(fingerprint_length):
        if f[i] == "1": fingerprint_cols[i].append(1)
        else: fingerprint_cols[i].append(0)

# CUT OFF COLUMNS WITH LOW VARIATION
min_spread = 0.001
for i in range(fingerprint_length):
    if min_spread <= (fingerprint_cols[i].count(1) / len(fingerprint_cols[i])) <= 1 - min_spread:
        X['fp_' + str(i)] = fingerprint_cols[i]

SCALER = 2000
X = X.copy()
X = X.drop('smiles', axis=1).drop('fingerprint', axis=1)
X['mw'] = np.log(X['mw'])
X['valence e'] = np.log(X['valence e'])
y = np.log(np.array(df_train['bp_K'].values / SCALER, dtype=float))
input_size = len(X.columns)
_X = X.copy()
_X['bp_K'] = np.log(df_train['bp_K'].values)
print(X.head())

X_FINGERPRINT = X

In [ ]:
# IMPORT DATASET - DESCRIPTORS
df_train = (pd.read_csv('/kaggle/input/smiles-boiling-point/bp_descriptors.csv'))

SMILES = df_train["smiles"]
X = df_train.drop('bp_K', axis=1).drop('smiles', axis=1)

# CUT OFF COLUMNS WITH LOW VARIATION
min_spread = 0.01
TO_DROP = []
for col in X.columns:
    if max(X[col].value_counts()) / len(X[col]) >= 1 - min_spread: 
        TO_DROP.append(col)

for col in TO_DROP:
    X = X.drop(col, axis=1)

print('Final Columns:', len(X.columns))

scaler = StandardScaler()

COLUMNS = X.columns
SCALER = 2000
X = X.copy().select_dtypes(include=['float64', 'float32', 'int64', 'int32'])
y = df_train['bp_K'].values / SCALER
#y = np.log(np.array(df_train['bp_K'].values / SCALER, dtype=float))
input_size = len(X.columns)
_X = X.copy()
_X['bp_K'] = df_train['bp_K'].values
print(X.head())

#for c in X_FINGERPRINT.columns.values:
#    if c not in _X.columns.values:
#        _X[c] = X_FINGERPRINT[c]

X = np.array(X.values, dtype=float)

# Molecular Descriptors

RDKit and Mordred were used to create 2048 2D and 3D descriptors for each molecule. Features that had a variance of less than 1% were dropped, resulting in a total of 1424 descriptors for training. The correlation of these features with the boiling point is shown below.

These descriptors are significantly more correlated with the boiling point than the features from the Morgan fingerprints. 


In [ ]:
numeric_cols = _X.select_dtypes(include=['int64', 'float64', 'float32', 'int32']).columns
numeric_cols = numeric_cols.drop('bp_K')
corr = _X[numeric_cols].corrwith(_X['bp_K']).sort_values(ascending=False)

plt.figure(figsize=(12,6))
sns.barplot(x=corr.index, y=corr.values)
plt.xticks(rotation=90)
plt.title("Correlation of numeric features with boiling point")
plt.xticks([], [])
plt.show()

In [ ]:
SMILES = df_train["smiles"]

y = np.log(df_train['bp_K'].values)
#y = df_train['bp_K'].values / SCALER

scaler = StandardScaler()
X = scaler.fit_transform(X)
SMILES_TO_DESCRIPTORS = dict()
for row, s in zip(X, SMILES):
    SMILES_TO_DESCRIPTORS[s] = row
train_x, val_x, train_y, val_y = train_test_split(X, y, test_size=0.1, random_state=42)

train_dataset = []
for a, b in zip(train_x, train_y):
    train_dataset.append((a,b))

val_dataset = []
for a, b in zip(val_x, val_y):
    val_dataset.append((a,b))

SMILES_TO_BP = dict()
for bp, s in zip(y, SMILES):
    SMILES_TO_BP[s] = bp


In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

xgb = XGBRegressor(
    n_estimators=100,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.9,
    colsample_bytree=0.9,
    n_jobs=-1
)

param_dist = {
    'n_estimators':[200,300,400,500],
    'max_depth': [3,4,5],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}


#model = XGBRegressor()
#xgb = RandomizedSearchCV(model, param_distributions=param_dist, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, n_iter=10, verbose=True)
xgb_model = xgb.fit(train_x, train_y)

print(xgb_model.score(train_x, train_y))
print(xgb_model.score(val_x, val_y))
# Parameter which gives the best results
#print(f"Best Hyperparameters: {xgb.best_params_}")
# Accuracy of the model after using best parameters
#print(f"Best Score: {xgb.best_score_}")

The total data was again split into a training and testing set in a 9:1 split and an XGBoost model was fitted to the training data. Its hyperparameters were tuned using a Randomised CV Search with 5-fold cross validation. 

The final model achieved a training score of 0.9891 and a testing score of 0.8156. This very high training score compared to the lower testing score could imply some overfitting for the model.
Several Random Forest Regressors were also fitted to the training data using between 10 to 50 estimators. These all achieved higher testing scores than the XGBoost model, with the top performance using only 10 estimators (training score: 0.9585; testing score: 0.8683).

In [ ]:
from sklearn.ensemble import RandomForestRegressor

def train_rf(n_estimators, train_x, train_y, random_state=0):
    rf_model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
    rf_model.fit(train_x, train_y)

    return rf_model

train_score = []
val_score = []
forest_size = [10, 20, 30, 40] # , 50, 60, 70]
for n in forest_size:
    print("=== n_estimators: " + str(n) + " ===")
    rf = train_rf(n, train_x, train_y)
    train_s = rf.score(train_x, train_y)
    val_s = rf.score(val_x, val_y)
    print(train_s, val_s)
    train_score.append(train_s)
    val_score.append(val_s)

In [ ]:
plt.plot(forest_size, train_score, label="training")
plt.plot(forest_size, val_score, label="validation")
plt.ylabel('Loss')
plt.xlabel('Forest Size')
plt.legend(loc='best')
plt.show()

In [ ]:
rf_model = train_rf(10, train_x, train_y) 
print("Random Forest Regressor - 10 Estimators")
print("Training Score: ", rf_model.score(train_x, train_y))
print("Validation Score: ", rf_model.score(val_x, val_y))

A plot of the prediction vs actual boiling points for the best performing Random Forest model is given below. Lines have been drawn showing the +/- 10% confidence interval. 

In [ ]:
chosen_model = rf_model

scatter_x = []
scatter_y = []
scatter_val_x = []
scatter_val_y = []
for molecule, bp in tqdm(zip(train_x, train_y), total=min(len(train_x), len(train_y))):
    molecule = molecule.reshape(1,-1)
    pred_bp = chosen_model.predict(molecule)
    scatter_x.append(pred_bp)
    scatter_y.append(bp)
for molecule, bp in tqdm(zip(val_x, val_y), total=min(len(val_x), len(val_y))):
    molecule = molecule.reshape(1,-1)
    pred_bp = chosen_model.predict(molecule)
    scatter_val_x.append(pred_bp)
    scatter_val_y.append(bp)

scatter_x = np.exp(scatter_x)
scatter_y = np.exp(scatter_y)
scatter_val_x = np.exp(scatter_val_x)
scatter_val_y = np.exp(scatter_val_y)

plt.scatter(scatter_x, scatter_y, label='training', s=5, alpha=0.5)
plt.scatter(scatter_val_x, scatter_val_y, label='validation', s=5, alpha=0.5)
plt.plot(scatter_y, scatter_y, c="black", linewidth=0.5)
plt.plot(scatter_y, 1.1* np.array(scatter_y), c="black", linestyle="dashed", linewidth=0.5)
plt.plot(scatter_y, 0.9* np.array(scatter_y), c="black", linestyle="dashed", linewidth=0.5)
plt.xlabel("Predicted BP / K")
plt.ylabel("Actual BP / K")
plt.legend(loc="best")
plt.show()

In [ ]:
molecules = ['CC(Cl)(Cl)Cl', 'O', 'N#Cc1ccccn1', 'C=CCN', 'CC(C)C', 'S=C=S', 'C']
molecules = ['SCCS', 'COC(CCl)=O', 'CCCCCCCC#CCCC', 'CCC[N+](=O)[O-]', 'CCC(C)C(SC1C)N=C1C']

for mol in molecules:
    x = SMILES_TO_DESCRIPTORS[mol]
    pred = np.exp(rf_model.predict([x])[0])
    bp = np.exp(SMILES_TO_BP[mol])
    diff = 100 * abs(bp - pred) / bp
    print(f'{mol}. Predicted: {pred:.3f}. Actual: {bp:.3f}. Difference {diff:.1f}%')

In [ ]:
DESCRIPTORS_TO_SMILES = dict()
for key, value in SMILES_TO_DESCRIPTORS.items():
    DESCRIPTORS_TO_SMILES[tuple(value)] = key
cnt = 0
for x in val_x:
    cnt += 1
    mol = DESCRIPTORS_TO_SMILES[tuple(x)]
    x = SMILES_TO_DESCRIPTORS[mol]
    pred = np.exp(rf_model.predict([x])[0])
    bp = np.exp(SMILES_TO_BP[mol])
    diff = 100 * abs(bp - pred) / bp
    print(f'{mol}. Predicted: {pred:.3f}. Actual: {bp:.3f}. Difference {diff:.1f}%')
    if cnt > 20: break

94.4% of all molecules in the training and testing datasets scored a percentage difference smaller than 10%. Meanwhile, 84.8% of all molecules scored a percentage difference smaller than 5%, and 51.1% of all molecules scored a percentage difference smaller than 1%.

In [ ]:
molecules = []
SMILES_SET = set(SMILES)
below_10 = 0
for mol in tqdm(SMILES_SET):
    x = SMILES_TO_DESCRIPTORS[mol]
    pred = np.exp(rf_model.predict([x])[0])
    bp = np.exp(SMILES_TO_BP[mol])
    diff = 100 * abs(bp - pred) / bp
    if diff < 10: below_10 += 1
    molecules.append((diff, pred, bp, mol))
molecules = sorted(molecules)[::-1]
for i in range(10):
    print(molecules[i])
print(below_10, below_10 / len(molecules))

# Model Insights

Permutation importance and SHAP values were used to uncover insights into the trained Random Forest Regressor. 

The SHAP force plots demonstrate how each descriptor impacted the final model prediction, examples of which are given below.

In [ ]:
feature_importances = pd.DataFrame({
    'feature': numeric_cols,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

# Get top 20 most important features
top_20_features = feature_importances.head(20)

# Visualize the results
plt.figure(figsize=(12, 8))
sns.barplot(x='importance', y='feature', data=top_20_features)
plt.title('Top 20 Feature Importances of XGBoost Model')
plt.xlabel('Importance')
plt.ylabel('Features')
plt.tight_layout()
plt.show()

In [ ]:
import shap

# Create object that can calculate shap values
explainer = shap.TreeExplainer(rf_model)

# Calculate Shap values
shap_values = explainer.shap_values(val_x)

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[2], val_x[2], feature_names=numeric_cols.values.tolist())

Permutation importance can be used to rank the importance of each individual feature in the model as a whole. The top 10 features and their relative weights are given below.

The most important descriptors identified by both the SHAP values and permutation importance are difficult to interpret as they correspond to abstract structural features of the molecules. 

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
perm = PermutationImportance(rf_model, random_state=1).fit(val_x, val_y)
eli5.show_weights(perm, feature_names = numeric_cols.values.tolist())

# Conclusion

SMILES representations of molecules can be used to generate molecular fingerprints and descriptors, both of which can be used to predict molecular properties with a reasonable accuracy. 

Molecular descriptors include more detailed information about the molecule and are more strongly correlated with boiling point. This translates to a more accurate performance on the testing dataset. 